In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# merged is already available from Step 3
# If not, run Step 3 first


# =========================================================
# 1) Extreme Fear vs Extreme Greed Performance
# =========================================================
extreme = merged[merged["classification"].isin(["Extreme Fear", "Extreme Greed"])].copy()

extreme_summary = extreme.groupby("classification").agg(
    total_trades=("Trade ID", "count"),
    total_pnl=("Closed PnL", "sum"),
    avg_pnl=("Closed PnL", "mean"),
    median_pnl=("Closed PnL", "median"),
    win_rate=("is_win", "mean"),
    avg_size_usd=("Size USD", "mean"),
    avg_fee=("Fee", "mean")
).reset_index()

extreme_summary["win_rate"] = (extreme_summary["win_rate"] * 100).round(2)

print("\nExtreme Fear vs Extreme Greed Summary:")
print(extreme_summary)

plt.figure(figsize=(7,4))
plt.bar(extreme_summary["classification"], extreme_summary["avg_pnl"])
plt.title("Average PnL: Extreme Fear vs Extreme Greed")
plt.ylabel("Average Closed PnL")
plt.show()


# =========================================================
# 2) Coin-wise performance: Fear vs Greed
# =========================================================
coin_summary = merged.groupby(["Coin", "sentiment_group"]).agg(
    trades=("Trade ID", "count"),
    total_pnl=("Closed PnL", "sum"),
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("is_win", "mean")
).reset_index()

coin_summary["win_rate"] = (coin_summary["win_rate"] * 100).round(2)

# Keep only coins with enough trades
coin_total = merged.groupby("Coin")["Trade ID"].count().sort_values(ascending=False)
top_coins = coin_total.head(10).index.tolist()

coin_summary_top = coin_summary[coin_summary["Coin"].isin(top_coins)]

print("\nTop 10 Coins (by trade count):")
print(top_coins)

print("\nCoin-wise Performance (Top 10 coins):")
print(coin_summary_top.sort_values(["Coin", "sentiment_group"]))

# Plot: Avg PnL per coin in Fear vs Greed
for coin in top_coins:
    temp = coin_summary_top[coin_summary_top["Coin"] == coin]
    plt.figure(figsize=(6,4))
    plt.bar(temp["sentiment_group"], temp["avg_pnl"])
    plt.title(f"Avg PnL by Sentiment for {coin}")
    plt.ylabel("Avg Closed PnL")
    plt.show()


# =========================================================
# 3) Top vs Bottom Traders (Behavior Differences)
# =========================================================
account_total_pnl = merged.groupby("Account")["Closed PnL"].sum().sort_values(ascending=False)

top_10pct_cutoff = int(len(account_total_pnl) * 0.10)
bottom_10pct_cutoff = int(len(account_total_pnl) * 0.10)

top_accounts = account_total_pnl.head(top_10pct_cutoff).index
bottom_accounts = account_total_pnl.tail(bottom_10pct_cutoff).index

merged["trader_group"] = np.where(
    merged["Account"].isin(top_accounts), "Top 10%",
    np.where(merged["Account"].isin(bottom_accounts), "Bottom 10%", "Middle 80%")
)

group_summary = merged[merged["trader_group"].isin(["Top 10%", "Bottom 10%"])].groupby(
    ["trader_group", "sentiment_group"]
).agg(
    trades=("Trade ID", "count"),
    total_pnl=("Closed PnL", "sum"),
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("is_win", "mean"),
    avg_size_usd=("Size USD", "mean"),
    avg_fee=("Fee", "mean")
).reset_index()

group_summary["win_rate"] = (group_summary["win_rate"] * 100).round(2)

print("\nTop 10% vs Bottom 10% Traders (Fear vs Greed):")
print(group_summary)

# Plot avg pnl for top vs bottom by sentiment
for grp in ["Top 10%", "Bottom 10%"]:
    temp = group_summary[group_summary["trader_group"] == grp]
    plt.figure(figsize=(6,4))
    plt.bar(temp["sentiment_group"], temp["avg_pnl"])
    plt.title(f"Avg PnL: {grp} Traders (Fear vs Greed)")
    plt.ylabel("Avg Closed PnL")
    plt.show()


# =========================================================
# 4) Final Insight Tables (Ready for Report)
# =========================================================
final_insights = merged.groupby("sentiment_group").agg(
    total_trades=("Trade ID", "count"),
    total_pnl=("Closed PnL", "sum"),
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("is_win", "mean"),
    avg_size_usd=("Size USD", "mean"),
    avg_fee=("Fee", "mean")
).reset_index()

final_insights["win_rate"] = (final_insights["win_rate"] * 100).round(2)

print("\nFinal Insight Table (Fear vs Greed):")
print(final_insights)


NameError: name 'merged' is not defined